# 第七章：使用示例（少样本提示）

本章将学习如何通过提供示例来改善Claude的响应质量和格式。这种技术被称为"少样本提示"，是提示工程中最有效的技术之一。

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装Anthropic库
%pip install anthropic

# 导入Python内置的正则表达式库，用于文本匹配和处理
import re
# 导入Anthropic库，用于与Claude API交互
import anthropic

# 从IPython存储中检索API密钥和模型名称变量
# 这些变量通常在之前的notebook中设置过
%store -r API_KEY
%store -r MODEL_NAME

# 创建Anthropic客户端实例，用于与Claude API通信
client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    """
    获取Claude的回复的辅助函数
    
    参数:
    - prompt: 用户输入的提示文本
    - system_prompt: 系统级提示，用于设置Claude的行为和角色
    - prefill: 预填充内容，为Claude的回复提供开始内容
    
    返回:
    - Claude的回复文本
    """
    message = client.messages.create(
        model=MODEL_NAME,           # 使用指定的模型
        max_tokens=2000,           # 设置最大回复长度
        temperature=0.0,           # 设置温度为0，确保结果的一致性和确定性
        system=system_prompt,      # 系统提示
        messages=[
          {"role": "user", "content": prompt},          # 用户消息
          {"role": "assistant", "content": prefill}     # 助手的预填充回复
        ]
    )
    return message.content[0].text

---

## 课程

**给Claude提供您希望它如何表现（或不希望它如何表现）的示例是极其有效的**，这种方法适用于：
- 获得正确的答案
- 以正确的格式获得答案

这种提示技术也被称为"**少样本提示**"（Few-shot Prompting）。您可能还会遇到"零样本"（zero-shot）、"n样本"（n-shot）或"单样本"（one-shot）这些术语。"样本"的数量指的是提示中使用了多少个示例。

**关键概念：**
- **零样本提示**：不提供任何示例，直接描述任务
- **单样本提示**：提供一个示例
- **少样本提示**：提供少量（通常2-5个）示例
- **多样本提示**：提供更多示例（但受限于上下文长度）

### 示例1：改变语调风格

假设您是一名开发者，试图构建一个回答孩子问题的"家长机器人"。**Claude的默认响应相当正式和机械化**，这会让孩子很失望。

让我们看看在没有示例指导的情况下，Claude如何回答儿童的问题：

In [ ]:
# 示例：默认响应风格
# 这个例子展示Claude默认的正式、机械化响应，不适合与儿童对话
PROMPT = "Will Santa bring me presents on Christmas?"

# 打印Claude的响应
print("=== Claude的默认响应（正式且机械化）===")
print(get_completion(PROMPT))
print("\n注意：这种回应对孩子来说太过正式和冷淡")

您可以花时间详细描述您想要的语调和风格，但**只给Claude一些理想响应的例子**会更容易、更有效。

通过示例，Claude可以：
- 理解期望的语调和风格
- 学习特定的回应模式
- 保持一致的输出格式
- 适应特定的用户群体（如儿童）

In [ ]:
# 改进版本：使用少样本提示
# 这个改进版本通过提供理想响应的示例来训练Claude的语调
# 注意示例中温暖、亲切的"家长"语调，适合与儿童对话
PROMPT = """请继续这个对话，以"A"的身份写下一行回复。
Q: 牙仙真的存在吗？
A: 当然存在，小宝贝。把你的牙齿包好放在枕头下面。明天早上可能会有惊喜等着你哦。
Q: 圣诞老人会给我带礼物吗？"""

# 打印Claude的响应
print("=== 使用示例后的温暖回应 ===")
print(get_completion(PROMPT))
print("\n注意：现在的回应更温暖、更适合儿童！")

### 示例2：格式化输出

在下面的格式化示例中，我们展示了两种方法：
1. **复杂方法**：逐步引导Claude完成一系列格式化指令，详细说明如何提取姓名和职业
2. **简单方法**：**只为Claude提供一些正确格式化的示例，让Claude从中推断规律**

第二种方法更简洁有效。注意我们在`assistant`轮次中使用`<individuals>`标签作为预填充，这为Claude提供了一个良好的开始格式。

In [ ]:
# 少样本提示示例：人物信息提取和格式化
# 这个复杂示例展示如何通过少样本提示让Claude学习特定的格式化模式
# 前两个段落提供了正确格式化的示例，第三个段落让Claude应用学到的模式
PROMPT = """银雾谷是一个迷人的村庄，这里住着一群非凡的人们。
其中有李明博士，他是一位神经外科医生，在地区医疗中心革新了手术技术。
陈雨薇是一位创新建筑师，用她可持续发展和令人惊叹的设计改变了村庄的景观。
当地剧院因为音乐家兼作曲家张浩天的迷人交响乐而熠熠生辉。
自学成才的厨师王小雅对当地食材充满热情，她的农场餐桌餐厅成为了美食爱好者的必访之地。
这些杰出的人物，各自拥有独特的才能，为银雾谷的生活增添了绚烂的色彩。
<individuals>
1. 李明博士 [神经外科医生]
2. 陈雨薇 [建筑师]
3. 张浩天 [音乐家兼作曲家]
4. 王小雅 [厨师]
</individuals>

在镇中心，厨师刘志强用他的农场餐桌餐厅"绿盘子"改变了当地的烹饪景象。刘志强致力于采购当地有机食材，为餐厅赢得了美食评论家和当地人的一致好评。
就在街道不远处，您会发现河边林图书馆，首席图书管理员赵丽华努力创造一个欢迎和包容的空间。她扩展图书馆服务并为儿童建立阅读项目的努力，对镇上的识字率产生了重大影响。
当您漫步在迷人的镇广场时，您会被墙上美丽的壁画所吸引。这些杰作是著名艺术家孙美琳的作品，她的才华将河边林镇的精髓展现得淋漓尽致。
河边林的体育成就也值得注意，这要归功于前奥运游泳运动员转为教练的马强。马强用他的经验和热情训练镇上的青年，带领河边林游泳队获得了多个地区冠军。
<individuals>
1. 刘志强 [厨师]
2. 赵丽华 [图书管理员]
3. 孙美琳 [艺术家]
4. 马强 [教练]
</individuals>

橡树谷是一个迷人的小镇，这里有三位杰出的人士，他们的技能和奉献精神对社区产生了深远的影响。
在镇上繁忙的农贸市场，您会找到热情的有机农民林小慧，她以美味和可持续种植的农产品而闻名。她对促进健康饮食的奉献激励了镇上拥抱更加环保的生活方式。
在橡树谷的社区中心，技艺娴熟的舞蹈教练李大伟为各个年龄段的人们带来了运动的快乐。他的包容性舞蹈课程在居民中培养了团结和自我表达的意识，丰富了当地的艺术景象。
最后，不知疲倦的志愿者王慧敏将她的时间奉献给各种慈善事业。她对改善他人生活的承诺对在橡树谷创造强烈的社区意识起到了重要作用。
通过他们独特的才能和坚定不移的奉献精神，林小慧、李大伟和王慧敏已经融入了橡树谷的结构中，帮助创造了一个充满活力和繁荣的小镇。"""

# Claude响应的预填充内容
# 使用<individuals>标签引导Claude按照前面示例的模式格式化输出
# 这是一个关键技巧：预填充给Claude一个明确的开始格式
PREFILL = "<individuals>"

# 打印完整的提示和Claude的响应
print("=== 完整提示内容 ===")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次预填充:")
print(PREFILL)
print("\n=== Claude按照示例模式的格式化输出 ===")
response = get_completion(PROMPT, prefill=PREFILL)
print(response)
print("\n注意：Claude学会了从文本中提取人名和职业，并按照示例的格式进行输出！")

如果您想要试验课程中的提示而不改变上面的任何内容，请滚动到笔记本的底部访问[**示例演练场**](#example-playground)。

**小结：**
- 少样本提示是一种强大的技术，可以显著改善Claude的输出质量
- 通过提供示例，我们可以教会Claude特定的语调、风格和格式
- 预填充技术可以引导Claude按照期望的格式开始回应
- 示例比详细的文字描述更有效

---

## 练习
- [练习 7.1 - 通过示例进行电子邮件格式化](#exercise-71---email-formatting-via-examples)

### 练习 7.1 - 通过示例进行电子邮件格式化
我们将重做练习6.2，但这次我们将编辑`PROMPT`，使用电子邮件的"少样本"示例+正确分类（和格式化）来让Claude输出正确答案。我们希望Claude输出的*最后*一个字母是类别的字母。

如果您忘记了每封邮件对应的正确字母类别，请参考`EMAILS`列表中每封邮件旁边的注释。

记住这些是电子邮件的类别：										
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请说明）								

In [ ]:
# 练习：电子邮件分类的少样本提示模板
# 任务：修改这个提示来包含少样本示例，展示正确的分类格式
# 目标：让Claude输出的最后一个字符是正确的类别字母（A、B、C或D）

# 当前的提示模板（需要改进）
# 这个提示太简单，没有提供任何关于电子邮件分类的信息
PROMPT = """请将这封电子邮件分类为以下类别之一：
(A) 售前问题
(B) 损坏或有缺陷的商品  
(C) 账单问题
(D) 其他（请说明）

电子邮件内容：{email}

请以字母结尾回复。"""

# Claude响应的预填充内容
# 可以在这里添加预填充来引导Claude的回复格式
PREFILL = ""

# 测试用的电子邮件列表
# 每封邮件后面的注释标明了正确的分类
EMAILS = [
    "您好 -- 我的搅拌器4000在使用时发出奇怪的噪音。它还有点烟雾和塑料味，像烧焦的电子产品。我需要更换。", # (B) 损坏或有缺陷的商品
    "我可以用搅拌器4000来搅拌油漆吗，还是只能用来搅拌食物？", # (A) 售前问题 或 (D) 其他（请说明）
    "我取消后已经等了4个月才停止月费扣款！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这里的，我不太会用电脑。求助。" # (D) 其他（请说明）
]

# 正确答案列表
# 存储为列表的列表，因为有些邮件可能有多个正确分类
ANSWERS = [
    ["B"],      # 第一封邮件的正确答案
    ["A","D"],  # 第二封邮件的正确答案（可以是A或D）
    ["C"],      # 第三封邮件的正确答案
    ["D"]       # 第四封邮件的正确答案
]

# 遍历所有测试邮件，评估分类效果
for i, email in enumerate(EMAILS):
    
    # 将具体的邮件内容填入模板的占位符中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取Claude的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 评分逻辑：检查Claude回复的最后一个字符是否是正确的分类字母
    # re.search()在字符串中搜索指定模式，response[-1]获取最后一个字符
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # 显示结果
    print(f"=== 测试邮件 {i+1} ===")
    print("邮件内容：")
    print(email)
    print(f"\n期望分类：{ANSWERS[i]}")
    print("\nClaude的回复：")
    print(response)
    print(f"\n最后字符：'{response[-1]}'")
    print(f"分类正确：{grade}")
    print("-" * 60)

❓ 如果您需要提示，请运行下面的单元格！

**练习指导：**
1. 当前的提示模板过于简单，Claude可能无法准确分类
2. 您需要在提示中添加少样本示例，展示每种类别的典型邮件
3. 确保示例清楚地展示了期望的输出格式（以分类字母结尾）
4. 可以考虑使用预填充来引导回复格式

In [ ]:
# 运行此单元格获取练习提示
# 如果hints模块不存在，我们提供一个自定义的提示
try:
    from hints import exercise_7_1_hint
    print(exercise_7_1_hint)
except ImportError:
    print("""
练习提示：

要改善电子邮件分类的准确性，您应该：

1. 在PROMPT中添加少样本示例，每个类别至少一个例子
2. 确保示例展示了完整的分类过程：邮件内容 → 分析 → 类别字母
3. 使用一致的输出格式，确保每个回复都以分类字母结尾
4. 可以考虑在PREFILL中添加引导词，如"分类："

示例格式：
邮件：[示例邮件内容]
分析：[简短分析为什么属于此类别]
分类：A

修改PROMPT变量以包含这样的示例！
""")

还是有困难？运行下面的单元格查看示例解决方案。

In [ ]:
# 运行此单元格查看练习的示例解决方案
# 如果hints模块不存在，我们提供一个自定义的解决方案
try:
    from hints import exercise_7_1_solution
    print(exercise_7_1_solution)
except ImportError:
    print("""
示例解决方案：

# 改进的提示模板，包含少样本示例
PROMPT = '''请根据以下示例将电子邮件分类：

示例1：
邮件：我想了解你们新产品的价格和功能特点。
分析：这是关于产品信息的咨询，属于售前问题。
分类：A

示例2：
邮件：我购买的耳机左声道没有声音，可能是质量问题。
分析：这是关于产品缺陷的报告，需要售后处理。
分类：B

示例3：
邮件：为什么我的信用卡被扣了两次同样的费用？
分析：这是关于重复扣费的账单问题。
分类：C

示例4：
邮件：请问你们公司的营业时间是什么？
分析：这是一般性咨询，不属于前面的具体类别。
分类：D

现在请分类这封邮件：
{email}

请按照上述格式回复，以分类字母结尾。'''

# 可选：使用预填充引导格式
PREFILL = "分析："

这样的改进应该能显著提高分类准确性！
""")

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示工程愉快！

---

## 示例演练场

这是一个供您自由试验本课程中展示的提示示例的区域，您可以调整提示来观察如何影响Claude的响应。

**实验建议：**
- 尝试修改示例的数量和质量
- 比较有示例和无示例的效果差异
- 测试不同的预填充策略
- 观察温度参数对少样本提示的影响

In [ ]:
# 实验1：基础提示（无示例）
# 这是没有任何示例指导的简单提示
PROMPT = "圣诞老人会给我带礼物吗？"

# 打印Claude的响应
print("=== 基础提示（无示例）的响应 ===")
print(get_completion(PROMPT))
print("\n注意：响应可能过于正式或成人化")

In [ ]:
# 实验2：单样本提示（一个示例）
# 通过提供一个温暖的示例对话来引导Claude的风格
PROMPT = """请继续这个对话，以"A"的身份写下一行回复。
Q: 牙仙真的存在吗？
A: 当然存在，小宝贝。把你的牙齿包好放在枕头下面。明天早上可能会有惊喜等着你哦。
Q: 圣诞老人会给我带礼物吗？"""

# 打印Claude的响应
print("=== 单样本提示的响应 ===")
print(get_completion(PROMPT))
print("\n注意：现在的响应更温暖、更适合儿童")

In [ ]:
# 实验3：多样本格式化提示（复杂示例）
# 这个实验展示如何通过多个示例教会Claude提取和格式化信息
PROMPT = """银雾谷是一个迷人的村庄，这里住着一群非凡的人们。
其中有李明博士，他是一位神经外科医生，在地区医疗中心革新了手术技术。
陈雨薇是一位创新建筑师，用她可持续发展和令人惊叹的设计改变了村庄的景观。
当地剧院因为音乐家兼作曲家张浩天的迷人交响乐而熠熠生辉。
自学成才的厨师王小雅对当地食材充满热情，她的农场餐桌餐厅成为了美食爱好者的必访之地。
这些杰出的人物，各自拥有独特的才能，为银雾谷的生活增添了绚烂的色彩。
<individuals>
1. 李明博士 [神经外科医生]
2. 陈雨薇 [建筑师]
3. 张浩天 [音乐家兼作曲家]
4. 王小雅 [厨师]
</individuals>

在镇中心，厨师刘志强用他的农场餐桌餐厅"绿盘子"改变了当地的烹饪景象。刘志强致力于采购当地有机食材，为餐厅赢得了美食评论家和当地人的一致好评。
就在街道不远处，您会发现河边林图书馆，首席图书管理员赵丽华努力创造一个欢迎和包容的空间。她扩展图书馆服务并为儿童建立阅读项目的努力，对镇上的识字率产生了重大影响。
当您漫步在迷人的镇广场时，您会被墙上美丽的壁画所吸引。这些杰作是著名艺术家孙美琳的作品，她的才华将河边林镇的精髓展现得淋漓尽致。
河边林的体育成就也值得注意，这要归功于前奥运游泳运动员转为教练的马强。马强用他的经验和热情训练镇上的青年，带领河边林游泳队获得了多个地区冠军。
<individuals>
1. 刘志强 [厨师]
2. 赵丽华 [图书管理员]
3. 孙美琳 [艺术家]
4. 马强 [教练]
</individuals>

橡树谷是一个迷人的小镇，这里有三位杰出的人士，他们的技能和奉献精神对社区产生了深远的影响。
在镇上繁忙的农贸市场，您会找到热情的有机农民林小慧，她以美味和可持续种植的农产品而闻名。她对促进健康饮食的奉献激励了镇上拥抱更加环保的生活方式。
在橡树谷的社区中心，技艺娴熟的舞蹈教练李大伟为各个年龄段的人们带来了运动的快乐。他的包容性舞蹈课程在居民中培养了团结和自我表达的意识，丰富了当地的艺术景象。
最后，不知疲倦的志愿者王慧敏将她的时间奉献给各种慈善事业。她对改善他人生活的承诺对在橡树谷创造强烈的社区意识起到了重要作用。
通过他们独特的才能和坚定不移的奉献精神，林小慧、李大伟和王慧敏已经融入了橡树谷的结构中，帮助创造了一个充满活力和繁荣的小镇。"""

# 预填充：为Claude提供格式化的开始
# 这告诉Claude应该以什么格式开始回复
PREFILL = "<individuals>"

# 打印实验结果
print("=== 多样本格式化提示实验 ===")
print("用户输入的文本片段：")
print("(显示最后一段)")
print("橡树谷是一个迷人的小镇...（省略具体内容）")
print(f"\n预填充引导：{PREFILL}")
print("\nClaude按照示例模式的输出：")
response = get_completion(PROMPT, prefill=PREFILL)
print(response)
print("\n注意：Claude学会了提取人名和职业，并按照前面示例的格式输出！")

# 额外实验：尝试不同的预填充
print("\n" + "="*60)
print("额外实验：尝试不同的预填充")
alternative_prefill = "总结："
print(f"使用预填充：{alternative_prefill}")
print("\n输出：")
print(get_completion(PROMPT, prefill=alternative_prefill))